In [4]:
import pandas as pd
from mootdx.quotes import Quotes

# 创建 Quotes 对象
client = Quotes.factory(market='std')

# 获取 510300 的近一周 1 分钟 K 线数据
# 这里假设一周是5个交易日，每天240个1分钟K线数据，总共是1200条数据
symbol = '510300'
data = client.bars(symbol=symbol, frequency=8, offset=1200)

# 转换为 DataFrame
df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['datetime'])

df


In [5]:
# 历史分时行情，可用
from mootdx.quotes import Quotes

client = Quotes.factory(market='std')
# data=client.minutes(symbol='000001', date='20100105')
# data=client.minutes(symbol='510300', date='20160108')
data=client.minutes(symbol='161226', date='20240110')
data

In [24]:
from mootdx.quotes import Quotes
import pandas as pd

# from QHDB.clickhouse.db import storage_client

client = Quotes.factory(market='std')
ret_list=[]
ret=None
code='159980'
index = 0
while True:
    data = client.bars(symbol=code, frequency=4, start=index * 800, offset=(1 + index) * 800)
    # data = client.bars(symbol='000776', frequency=4, start=index * 800, offset=(1 + index) * 800)

    # print(data.to_string())
    # # 前复权
    # data = client.bars(symbol='600036', adjust='qfq')
    #
    # # 后复权
    # data = client.bars(symbol='600036', adjust='hfq')
    ret_list.append(data)
    if data.empty or len(data) < 800:
        break
    index += 1

if len(ret_list)>1:
    # 使用concat函数拼接DataFrame
    result_df = pd.concat(ret_list)

    # print(result_df)

    # 按照索引（datetime）重新排序
    result_df = result_df.sort_index()
    result_df.rename(columns={'datetime': 'date'}, inplace=True)
    # result_df.set_index('date', inplace=True)
    result_df["date"]=pd.to_datetime(result_df["date"])
    selected_columns = ['open', 'close', 'high', 'low', 'volume', 'amount','date',"code"]
    result_df["code"]=code
    result_df = result_df.loc[:, selected_columns]
    # print(result_df.to_string())
    ret=result_df
else:
    # 按照索引（datetime）重新排序
    ret_list[0] = ret_list[0].sort_index()
    ret_list[0].rename(columns={'datetime': 'date'}, inplace=True)
    # ret_list[0].set_index('date', inplace=True)
    ret_list[0]["date"]=pd.to_datetime(ret_list[0]["date"])
    selected_columns = ['open', 'close', 'high', 'low', 'volume', 'amount','date',"code"]
    ret_list[0]["code"]=code
    result_df = ret_list[0].loc[:, selected_columns]
    # print(ret_list[0].to_string())
    ret=ret_list[0]

rr=[]
for i in range((len(ret)-1)//95+1):
    end=len(ret) if (i+1)*95>(len(ret)-1) else (i+1)*95
    # storage_client.insert_dataframe('INSERT INTO tdx.bars_1d VALUES', ret[i*95:end])
    rr.append( ret[i*95:end])

rrr=pd.concat(rr)
# print(len(ret),len(rrr))
rrr

In [26]:
rrr_save=rrr[['date','open','close','high','low','volume','amount']]
rrr_save.drop(['amount'],axis=1,inplace=True)
rrr_save.rename(columns={'date':'datetime'},inplace=True)
rrr_save

In [27]:
rrr_save.to_csv('159980-day.csv',index=False)